In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import matplotlib.pyplot as plt
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#from grabscreen import grab_screen
import cv2
import glob
import random 
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from skimage.metrics import structural_similarity as ssim

In [2]:
import time

In [4]:
sys.path.append('D:/Github/tf_object_detection/models/research/object_detection')

In [ ]:
#https://pythonprogramming.net/detecting-distances-self-driving-car/
#https://medium.com/@marklabinski/installing-tensorflow-object-detection-api-on-windows-10-7a4eb83e1e7b

In [ ]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")



In [5]:

# ## Object detection imports
# Here are the imports from the object detection module.

from utils import label_map_util
from utils import visualization_utils as vis_util


In [6]:
# # Model preparation 
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
#PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
PATH_TO_LABELS = MODEL_NAME + '/mscoco_label_map.pbtxt'
NUM_CLASSES = 90


In [ ]:
# ## Download Model
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())


In [7]:
# ## Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


In [8]:
# ## Loading label map
# Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [9]:
# ## Helper code
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def distance_to_camera(predicted_Width):
    FOCAL_LENGTH = 841
    KNOWN_WIDTH = 1.5
    # compute and return the distance from the maker to the camera
    return (KNOWN_WIDTH * FOCAL_LENGTH) / predicted_Width
def checkIfpointIsinPolygone(point,poly):
    return poly.contains(point)

def getVehicleSpeed(ssimScore):
    
    if ssimScore >=0.98:
        return 0
    
    setSpeed = 60
    calculatedSpeed = setSpeed * ssimScore
    speed = setSpeed-calculatedSpeed
    return int(speed)
       
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [10]:
%matplotlib inline
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        frameNumber = 0
        vid = cv2.VideoCapture('vid1.mp4')
        frame_width = int(vid.get(3))
        frame_height = int(vid.get(4))
        out = cv2.VideoWriter('outpy.mp4',cv2.VideoWriter_fourcc('F','M','P','4'), 10, (frame_width,frame_height))
        while True:
            
            #screen = cv2.resize(grab_screen(region=(0,40,1280,745)), (WIDTH,HEIGHT))
            #vid = cv2.VideoCapture('1.mp4')
            #ret, frame = vid.read()
        #images = glob.glob("./images/*.png")
        #for image in images:
            start_time = time.time() # start time of the loop
            #img = cv2.imread(image)
            ret, img = vid.read()
           
            
           
            
                
            #img = cv2.imread('./images/1.png')
            screen = cv2.resize(img, (800,450))
            image_np = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
            height, width, channels = image_np.shape
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')


            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run([boxes, scores, classes, num_detections],
                      feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=3)

            #polygone = np.array([[[0, height] , [width, height], [round(width/1.17), round(height/2)],[(width-round(width/1.17)), round(height/2)],]], np.int32)
            
            #cv2.polylines(image_np, [polygone], True, (0,255,0), thickness=1)
            poly_scale = 1.7
            polygone = np.array([[[0, height] , [width, height], [round(width/poly_scale), round(height/poly_scale)],[(width-round(width/poly_scale)), round(height/poly_scale)],]], np.int32)
            polygonePoints = Polygon([(0, height),(width, height),(round(width/poly_scale), round(height/poly_scale)),((width-round(width/poly_scale)), round(height/poly_scale))])
            cv2.putText(image_np,'Vehicle Speed :', (550,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (102,255,102), 2)
            cv2.putText(image_np,'Collision Time :', (550,44), cv2.FONT_HERSHEY_SIMPLEX, 0.6,(102,255,102), 2)
            cv2.putText(image_np,'Breaking Flag :', (550,66), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (102,255,102), 2)
            
            
            
            
            
            #vehicle speed calculations
            if frameNumber ==0:
                oldFrame = image_np
            #print('frameno',frameNumber)
            #print('old frame',oldFrame.shape)
            #print('new frame',image_np.shape)
            grayImage = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
            grayOldFrame = cv2.cvtColor(oldFrame, cv2.COLOR_BGR2GRAY)
           
            ssimScore = ssim(grayImage, grayOldFrame)
            oldFrame = image_np
            vehicleSpeed = getVehicleSpeed(ssimScore)
            cv2.putText(image_np,str(vehicleSpeed)+'Kmph', (720,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (102,255,102), 2)
            
            
            #polygone = np.array([[[0, height] , [width, height], [round(width/1.17), round(height/2)],[(width-round(width/1.17)), round(height/2)],]], np.int32)
            #image = cv2.rectangle(image, start_point, end_point, color, thickness)
            cv2.polylines(image_np, [polygone], True, (12,236,236), thickness=2)

            for i,b in enumerate(boxes[0]):
                #                 car                    bus                  truck
                if classes[0][i] == 3 or classes[0][i] == 6 or classes[0][i] == 8:
                    if scores[0][i] >= 0.5:
                        
                        #calculate distance of object here
                        bbox = boxes[0][i]
                        width_of_bounding_box = bbox[3]*width - bbox[1]*width
                        distance = round( distance_to_camera(width_of_bounding_box))
                        #print('distance from camera in meters = ',distance)
                        
                        ymin, xmin, ymax, xmax = bbox 
                        (left, right, top, bottom) = (xmin * width, xmax * width, 
                              ymin * height, ymax * height)
                        
                        start_point = (round(left),round(top))
                        end_point = (round(right),round(bottom))
                        color = (0,255,0)
                        thickness = 5
                        poly_scale = 1.7
                        
                        
                        
                        
                        
                        
                        #print(round(left), round(right), round(top), round(bottom))
                        
                        mid_x = (boxes[0][i][1]+boxes[0][i][3])/2
                        mid_y = (boxes[0][i][0]+boxes[0][i][2])/2
                    
                        
                        
                        
                        
                        
                        
                        apx_distance = round(((1 - (boxes[0][i][3] - boxes[0][i][1]))**4),1)
                        #cv2.putText(image_np, '{}'.format(apx_distance), (int(mid_x*800),int(mid_y*450)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

                        if apx_distance <=0.5:
                            
                            #print(round(left), round(right), round(top), round(bottom))
                            start_point = (round(left), round(top))
                            end_point = ( round(right),round(bottom))
                            #shoe bounding boxes
                            
                            #cv2.putText(image_np,'car', (round(left),round(top)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 2)
                            
                            cv2.putText(image_np, str(distance)+'m', (round(right),round(bottom)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (102,255,102), 2)
                            #if mid_x > 0.3 and mid_x < 0.8:
                            #check if the x_mid,y_mid are in the polygone
                            midPoint = Point(round(mid_x*width),round(mid_y*height))
                            flag =checkIfpointIsinPolygone(midPoint,polygonePoints)
                            if flag == True:   
                                cv2.putText(image_np, 'WARNING!!!', (330,100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0), 3)
                                image_np = cv2.rectangle(image_np, (323,67), (503,107), (102,255,102), 2)
                                cv2.polylines(image_np, [polygone], True, (255,0,0), thickness=2)   
                                image_np = cv2.rectangle(image_np, start_point, end_point, (255,0,0), 2)
                                cv2.putText(image_np, str(distance)+'m', (round(right),round(bottom)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
                                if vehicleSpeed != 0:    
                                    timeForCollision =round((distance/((vehicleSpeed*1000)/3600)),2)
                                    cv2.putText(image_np,str(timeForCollision) + 'sec', (720,44), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (102,255,102), 2)
                                cv2.putText(image_np,'True', (720,66), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (102,255,102), 2)
                            #cv2.polylines(image_np, [polygone], True, (0,255,0), thickness=2)
            im_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)               
            #cv2.imwrite(image,im_rgb)
            
            frameNumber =frameNumber+1
            cv2.imshow('window',cv2.resize(im_rgb,(800,450)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break
        #plt.imshow(image_np)
        print("FPS: ", 1.0 / (time.time() - start_time)) # FPS = 1 / time to process loop
            

FPS:  4.41762836682827


In [16]:
from os.path import exists

file_exists = exists(PATH_TO_CKPT) and exists(PATH_TO_LABELS)
file_exists

False